In [140]:
import pandas as pd
import yaml
import chardet
import ujson


In [139]:
conda install ujson

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.7.10
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base conda



## Package Plan ##

  environment location: /vortexfs1/home/halexander/.conda/envs/EUKulele

  added / updated specs:
    - ujson


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ujson-1.35                 |py37h14c3975_1001          27 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          27 KB

The following NEW packages will be INSTALLED:

  ujson              conda-forge/linux-64::ujson-1.35-py37h14c3975_1001



ujson-1.35           | 27 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done

Note: you may need to restart

In [76]:
def tax_placement(pident):
    if pident >= tax_cutoffs['species']:
        out = 'species'
    elif pident >= tax_cutoffs['genus']:
        out = 'genus'
    elif pident >= tax_cutoffs['family']:
        out = 'family'
    elif pident >= tax_cutoffs['order']:
        out = 'order'
    elif pident < tax_cutoffs['order']:
        out = 'class'
    return out

In [87]:
#Load Taxonomy File

tax_file = '/vortexfs1/omics/alexander/data/mmetsp/reference_dir/taxonomy-table.txt'
with open(tax_file, 'rb') as f:
    result = chardet.detect(f.read())  # or readline if the file is large
tax_table = pd.read_csv(tax_file, sep='\t',encoding=result['encoding'])
tax_table.columns = tax_table.columns.str.lower()
tax_table=tax_table.set_index('source_id')
#Read in tax-cutoffs
with open("tax-cutoffs.yaml", 'r') as stream:
    tax_cutoffs = yaml.safe_load(stream)


In [88]:
tax_table

,cyverse_path,source_id_dup,ref_status,strain,supergroup,division,class,order,family,genus,species,notes
source_id,,,,,,,,,,,,
MMETSP0290,2225.0,MMETSP0290,Clean,CCMP2878,Alveolata,Apicomplexa,Colpodellidea,Colpodellida,Chromeraceae,Chromera,Chromera velia,Curated by J. del Campo (PR2)
MMETSP0288,2224.0,MMETSP0288,Clean,CCMP3155,Alveolata,Apicomplexa,Colpodellidea,Vitrelladida,Vitrellaceae,Vitrella,Vitrella brassicaformis,Adl et al. 2019; Curated by J. del Campo and V...
MMETSP1451,2282.0,MMETSP1451,Clean,CCMP3346,Alveolata,Apicomplexa,Colpodellidea,Vitrelladida,Vitrellaceae,Vitrella,Vitrella brassicaformis,Adl et al. 2019; Curated by J. del Campo and V...
MMETSP0372,1822.0,MMETSP0372,Clean,Grappler Inlet BC,Alveolata,Apicomplexa,Gregarinomorphea,Eugregarinorida,Lecudinidae,Lankesteria,Lankesteria abbottii,Adl et al. 2019; Curated by J. del Campo (PR2)
MMETSP0125,1740.0,MMETSP0125,Clean,ATCC 50986,Alveolata,Ciliophora,Colpodea,Colpodea_X,Cyrtolophosidida,Aristerostoma,Aristerostoma sp.,Ciliophora EukRef curation Boscaro V.; previou...
...,...,...,...,...,...,...,...,...,...,...,...,...
MMETSP0990,1938.0,MMETSP0990,Clean,CCMP2098,Stramenopiles,Ochrophyta,Dictyochophyceae,Dictyochophyceae_X,Dictyochophyceae_XX,Uncertain,Uncertain,Not described; listed as Dictyochophyceae
MMETSP0991,1943.0,MMETSP0991,Clean,CCMP2098,Stramenopiles,Ochrophyta,Dictyochophyceae,Dictyochophyceae_X,Dictyochophyceae_XX,Uncertain,Uncertain,Not described; listed as Dictyochophyceae
MMETSP0992,1944.0,MMETSP0992,Clean,CCMP2098,Stramenopiles,Ochrophyta,Dictyochophyceae,Dictyochophyceae_X,Dictyochophyceae_XX,Uncertain,Uncertain,Not described; listed as Dictyochophyceae


In [145]:
%%timeit
prot_file = '/vortexfs1/omics/alexander/data/mmetsp/reference_dir/protein-table.txt'
pdict = {}
with open(prot_file, 'r')as f:
    for line in f:
        protein, mmetsp = line.split('\t')
        pdict[protein]=mmetsp.strip()


9.59 s ± 39.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [135]:
output_table = 'output/METs/diamond/test2.diamond.out'
df = pd.read_csv(output_table, sep = '\t', header = None)
df.columns = ['qseqid', 'sseqid', 'pident', 'length', 'mismatch', 'gapopen', 'qstart', 'qend', 'sstart', 'send', 'evalue', 'bitscore']
df['ssqidMMETSP']=df.sseqid.map(pdict)

In [136]:
classification_df = pd.DataFrame(columns = ['classification_level', 'classification', 'max_pid'])
for t,dd in df.groupby('qseqid'):
    md = dd.pident.max()
    ds = list(set(dd[dd.pident==md]['ssqidMMETSP']))
    assignment = tax_placement(md)
    if len(ds)==1:
        classification = tax_table.loc[ds[0], assignment]
    else:
        classification_0 = []
        for d in ds:
            classification_0.append(tax_table.loc[ds[0], assignment])
            if len(set(classification_0)) ==1:
                classification = list(set(classification_0))[0]
            else:
                pass
    classification_df.loc[t] = [assignment, classification, md]
   

In [137]:
classification_df.sort_values('max_pid')

,classification_level,classification,max_pid
GGNT01000149.1,class,Heterotrichea,23.6
GGNT01000126.1,class,Heterotrichea,23.8
GGNT01000448.1,class,Chlorarachniophyceae,24.5
GGNT01000065.1,class,Heterotrichea,24.6
GGNT01000482.1,class,Heterotrichea,24.8
...,...,...,...
GGNT01000452.1,family,Protocruziidae,75.6
GGNT01000318.1,family,Strombidinopsidae,76.3
GGNT01000025.1,family,Fabreidae,76.5
GGNT01000253.1,genus,Climacostomum,88.9


In [98]:
for t,dd in df.groupby('qseqid'):
    dd

In [150]:
mfi = '/vortexfs1/omics/alexander/data/mmetsp/reference_dir/protein-species-map.json'
with open(mfi, 'rb') as fp:
    imp = ujson.load(fp)


In [152]:
imp

{'CAMPEP_0174977180': 'MMETSP0004',
 'CAMPEP_0174977186': 'MMETSP0004',
 'CAMPEP_0174977190': 'MMETSP0004',
 'CAMPEP_0174977200': 'MMETSP0004',
 'CAMPEP_0174977208': 'MMETSP0004',
 'CAMPEP_0174977214': 'MMETSP0004',
 'CAMPEP_0174977224': 'MMETSP0004',
 'CAMPEP_0174977226': 'MMETSP0004',
 'CAMPEP_0174977234': 'MMETSP0004',
 'CAMPEP_0174977244': 'MMETSP0004',
 'CAMPEP_0174977246': 'MMETSP0004',
 'CAMPEP_0174977252': 'MMETSP0004',
 'CAMPEP_0174977256': 'MMETSP0004',
 'CAMPEP_0174977266': 'MMETSP0004',
 'CAMPEP_0174975610': 'MMETSP0004',
 'CAMPEP_0174977704': 'MMETSP0004',
 'CAMPEP_0174977706': 'MMETSP0004',
 'CAMPEP_0174977710': 'MMETSP0004',
 'CAMPEP_0174977728': 'MMETSP0004',
 'CAMPEP_0174977730': 'MMETSP0004',
 'CAMPEP_0174977744': 'MMETSP0004',
 'CAMPEP_0174977746': 'MMETSP0004',
 'CAMPEP_0174977754': 'MMETSP0004',
 'CAMPEP_0174977758': 'MMETSP0004',
 'CAMPEP_0174977764': 'MMETSP0004',
 'CAMPEP_0174977768': 'MMETSP0004',
 'CAMPEP_0174977770': 'MMETSP0004',
 'CAMPEP_0174977776': 'MMETS